In [1]:
!git clone -b prune-experiment https://github.com/atikul-islam-sajib/TreeBasedModel.git

Cloning into 'TreeBasedModel'...
remote: Enumerating objects: 988, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 988 (delta 48), reused 44 (delta 18), pack-reused 907
Receiving objects: 100% (988/988), 124.17 MiB | 10.80 MiB/s, done.
Resolving deltas: 100% (550/550), done.


In [2]:
%cd /content/TreeBasedModel

/content/TreeBasedModel


In [3]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 7.0 MB/s eta 0:00:00


In [4]:
!pip install -e . --verbose

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/TreeBasedModel
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info
  writing /tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-sqqzr1jc/AugmentedHierarchicalShrinkage.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  writing manifest file '/tmp/pip-pip-e

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest

# Generate synthetic dataset
X, y = make_classification(n_samples=200, n_features=20, n_informative=10, n_redundant=10, n_clusters_per_class=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Instantiate RandomForest
rf = RandomForest(n_trees=100, max_depth=10, min_samples_split=2, min_samples_leaf=1,
                  n_feature="sqrt", bootstrap=True, oob=True, criterion="gini",
                  treetype="classification", random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Function to write details to a file
def write_tree_details_to_file(filename, rf, stage):
    with open(filename, 'a') as file:
        file.write(f"\nTree structure {stage} pruning:\n")
        file.write(f"Tree depth {stage} pruning: {rf.trees[0].max_depth_}\n")

        for i, tree in enumerate(rf.trees):
            file.write(f"\nTree {i} {stage} pruning:\n")
            for node_id, node_info in tree.node_id_dict.items():
                if node_info['is_leaf_node']:
                    file.write(f"Node ID: {node_id}, Depth: {node_info['depth']}, Samples: {node_info['samples']}\n")

# Write details before pruning
write_tree_details_to_file('tree_details.txt', rf, 'before')

# Prune the forest with a higher min_samples_leaf value to force more pruning
rf.prune(min_samples_leaf=30)

# Write details after pruning
write_tree_details_to_file('tree_details.txt', rf, 'after')

# Evaluate the model
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Write evaluation results to the file
with open('tree_details.txt', 'a') as file:
    file.write(f'\nAccuracy: {accuracy:.2f}\n')
    file.write(f'OOB Score: {rf.oob_score:.2f}\n')

# Optionally, print the evaluation results to the console
print(f'\nAccuracy: {accuracy:.2f}')
print(f'OOB Score: {rf.oob_score:.2f}')


Accuracy: 0.80
OOB Score: 0.82


In [19]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest

# Generate synthetic dataset
X, y = make_classification(n_samples=200, n_features=20, n_informative=10, n_redundant=10, n_clusters_per_class=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Instantiate RandomForest
rf = RandomForest(n_trees=100, max_depth=10, min_samples_split=2, min_samples_leaf=1,
                  n_feature="sqrt", bootstrap=True, oob=True, criterion="gini",
                  treetype="classification", random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Function to write details to a file
def write_tree_details_to_file(filename, rf, stage):
    with open(filename, 'a') as file:
        file.write(f"\nTree structure {stage} pruning:\n")
        file.write(f"Tree depth {stage} pruning: {rf.trees[0].max_depth_}\n")

        for i, tree in enumerate(rf.trees):
            file.write(f"\nTree {i} {stage} pruning:\n")
            for node_id, node_info in tree.node_id_dict.items():
                if node_info['is_leaf_node']:
                    file.write(f"Node ID: {node_id}, Depth: {node_info['depth']}, Samples: {node_info['samples']}\n")

# Write details before pruning
write_tree_details_to_file('tree_details.txt', rf, 'before')

# Prune the forest with a higher min_samples_leaf value to force more pruning
rf.prune(min_samples_leaf=30)

# Write details after pruning
write_tree_details_to_file('tree_details.txt', rf, 'after')

# Check for low sample sizes after pruning
def check_low_sample_sizes(rf, threshold=30):
    low_sample_nodes = 0
    for tree in rf.trees:
        for node_id, node_info in tree.node_id_dict.items():
            if node_info['is_leaf_node'] and node_info['samples'] < threshold:
                low_sample_nodes += 1
    return low_sample_nodes

low_sample_nodes_count = check_low_sample_sizes(rf)

# Evaluate the model
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Write evaluation results to the file
with open('tree_details.txt', 'a') as file:
    file.write(f'\nAccuracy: {accuracy:.2f}\n')
    file.write(f'OOB Score: {rf.oob_score:.2f}\n')
    file.write(f'Number of leaf nodes with low sample sizes after pruning: {low_sample_nodes_count}\n')

# Optionally, print the evaluation results to the console
print(f'\nAccuracy: {accuracy:.2f}')
print(f'OOB Score: {rf.oob_score:.2f}')
print(f'Number of leaf nodes with low sample sizes after pruning: {low_sample_nodes_count}')


Accuracy: 0.80
OOB Score: 0.82
Number of leaf nodes with low sample sizes after pruning: 2961


### Check `Number of leaf nodes with low sample sizes`

In [24]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from TreeModelsFromScratch.RandomForest import RandomForest

# Generate synthetic dataset
X, y = make_classification(n_samples=200, n_features=20, n_informative=10, n_redundant=10, n_clusters_per_class=2, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Function to print tree details and debug information
def print_tree_details(rf, stage):
    print(f"\nTree structure {stage} pruning:")
    print(f"Tree depth {stage} pruning: {rf.trees[0].max_depth_}")

    for i, tree in enumerate(rf.trees):
        print(f"\nTree {i} {stage} pruning:")
        for node_id, node_info in tree.node_id_dict.items():
            if node_info['is_leaf_node']:
                print(f"Node ID: {node_id}, Depth: {node_info['depth']}, Samples: {node_info['samples']}")
            else:
                print(f"Node ID: {node_id}, Feature: {node_info['feature']}, Threshold: {node_info['threshold']}")

# Check for low sample sizes
def check_low_sample_sizes(rf, threshold=10):
    low_sample_nodes = 0
    for tree in rf.trees:
        for node_id, node_info in tree.node_id_dict.items():
            if node_info['is_leaf_node'] and node_info['samples'] < threshold:
                low_sample_nodes += 1
    return low_sample_nodes

# Evaluate with a single pruning parameter
min_samples_leaf = 15  # Trying a different value for min_samples_leaf

# Instantiate RandomForest with a single tree
rf = RandomForest(n_trees=1, max_depth=10, min_samples_split=2, min_samples_leaf=1,
                  n_feature="sqrt", bootstrap=True, oob=True, criterion="gini",
                  treetype="classification", random_state=42)

# Fit the model
rf.fit(X_train, y_train)

# Print details before pruning
print_tree_details(rf, 'before')

# Check for low sample sizes before pruning
low_sample_nodes_before = check_low_sample_sizes(rf)
print(f'Number of leaf nodes with low sample sizes before pruning: {low_sample_nodes_before}')

# Evaluate the model before pruning
y_pred_before = rf.predict(X_test)
accuracy_before = accuracy_score(y_test, y_pred_before)
print(f'Accuracy before pruning: {accuracy_before:.2f}')
print(f'OOB Score before pruning: {rf.oob_score:.2f}')

# Prune the forest with the chosen min_samples_leaf value
rf.prune(min_samples_leaf=min_samples_leaf)

# Print details after pruning
print_tree_details(rf, 'after')

# Check for low sample sizes after pruning
low_sample_nodes_after = check_low_sample_sizes(rf)
print(f'Number of leaf nodes with low sample sizes after pruning: {low_sample_nodes_after}')

# Evaluate the model after pruning
y_pred_after = rf.predict(X_test)
accuracy_after = accuracy_score(y_test, y_pred_after)

# Print evaluation results to the console
print(f'\nAccuracy after pruning: {accuracy_after:.2f}')
print(f'OOB Score after pruning: {rf.oob_score:.2f}')


Tree structure before pruning:
Tree depth before pruning: 7

Tree 0 before pruning:
Node ID: 0, Feature: 13, Threshold: -3.075385119347575
Node ID: 1, Feature: 7, Threshold: 0.4958788056715076
Node ID: 2, Feature: 12, Threshold: 10.913006457087866
Node ID: 3, Feature: 17, Threshold: 4.17591417936568
Node ID: 4, Depth: 4, Samples: 30
Node ID: 5, Feature: 12, Threshold: 0.7450495738099908
Node ID: 6, Depth: 5, Samples: 2
Node ID: 7, Depth: 5, Samples: 1
Node ID: 8, Depth: 3, Samples: 1
Node ID: 9, Feature: 1, Threshold: -0.21910673520782475
Node ID: 10, Depth: 3, Samples: 1
Node ID: 11, Depth: 3, Samples: 2
Node ID: 12, Feature: 10, Threshold: -0.5857705585028377
Node ID: 13, Feature: 16, Threshold: -4.704136377899882
Node ID: 14, Feature: 18, Threshold: 0.3606578420750475
Node ID: 15, Depth: 4, Samples: 1
Node ID: 16, Depth: 4, Samples: 5
Node ID: 17, Feature: 18, Threshold: 3.051315739814543
Node ID: 18, Depth: 4, Samples: 32
Node ID: 19, Depth: 4, Samples: 1
Node ID: 20, Feature: 3, 

/content/TreeBasedModel/TreeModelsFromScratch/RandomForest.py:277: UserWarning: 90 out of 150 samples do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates. These samples were dropped before computing the oob_score
  warn(message)
